In [1]:
# Update sklearn to prevent version mismatches
!pip install sklearn --upgrade

In [2]:
# install joblib. This will be used to save your model. 
# Restart your kernel after installing 
!pip install joblib

In [4]:
!pip install tensorflow

In [ ]:
# Set the seed value for the notebook so the results are reproducible
from numpy.random import seed
seed(1)

In [5]:
# Dependencies
import numpy as np
import pandas as pd

In [6]:
import tensorflow
tensorflow.keras.__version__

'2.2.4-tf'

In [7]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler ,OneHotEncoder
from tensorflow.keras.utils import to_categorical

# Read the CSV and Perform Basic Data Cleaning

In [8]:
#https://www.tensorflow.org/datasets/catalog/titanic
df_tensor = pd.read_csv("data/TensorFlowtitanic.csv")
# Drop the null columns where all values are null
df_tensor =df_tensor.dropna(axis='columns', how='all')
# Drop the null rows
df_tensor = df_tensor.dropna()
df_tensor.head()

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
0,1,1,"Allen, Miss. Elisabeth Walton",female,29,0,0,24160,211.3375,B5,S,2,?,"St Louis, MO"
1,1,1,"Allison, Master. Hudson Trevor",male,0.9167,1,2,113781,151.55,C22 C26,S,11,?,"Montreal, PQ / Chesterville, ON"
2,1,0,"Allison, Miss. Helen Loraine",female,2,1,2,113781,151.55,C22 C26,S,?,?,"Montreal, PQ / Chesterville, ON"
3,1,0,"Allison, Mr. Hudson Joshua Creighton",male,30,1,2,113781,151.55,C22 C26,S,?,135,"Montreal, PQ / Chesterville, ON"
4,1,0,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",female,25,1,2,113781,151.55,C22 C26,S,?,?,"Montreal, PQ / Chesterville, ON"


In [9]:
# Drop the null columns where all values are null
df_tensor = df_tensor.dropna(axis='columns', how='all')
# Drop the null rows
df_tensor = df_tensor.dropna()

In [10]:
df_tensor.count()

pclass       1309
survived     1309
name         1309
sex          1309
age          1309
sibsp        1309
parch        1309
ticket       1309
fare         1309
cabin        1309
embarked     1309
boat         1309
body         1309
home.dest    1309
dtype: int64

In [ ]:
df_tensor.columns

In [11]:
X = df_tensor.drop(["survived", 'parch', 'ticket', 'embarked' ,'name','home.dest','boat','body','cabin'], axis=1)

y= df_tensor["survived"]
print(X.shape, y.shape)

(1309, 5) (1309,)


In [ ]:
X


In [ ]:
y


In [12]:
X['sex'] = X['sex'].replace(to_replace='?', value=0)

In [ ]:
X


In [13]:
X['age'] = X['age'].replace(to_replace='?', value=0)

X['cabin'] = X['cabin'].replace(to_replace='?', value=0)

In [14]:
X['fare'] = X['fare'].replace(to_replace='?', value=0)

In [15]:
X['sibsp'] = X['sibsp'].replace(to_replace='?', value=0)

In [16]:
X['pclass'] = X['pclass'].replace(to_replace='?', value=0)

In [ ]:
X


In [17]:
data = X.copy()
data_binary_encoded = pd.get_dummies(data, columns=["sex"])
data_binary_encoded.head()

,pclass,age,sibsp,fare,sex_female,sex_male
0,1,29,0,211.3375,1,0
1,1,0.9167,1,151.55,0,1
2,1,2,1,151.55,1,0
3,1,30,1,151.55,0,1
4,1,25,1,151.55,1,0


In [18]:
new_data = data_binary_encoded.drop(["sex_female"],axis=1)

In [19]:
new_data

,pclass,age,sibsp,fare,sex_male
0,1,29,0,211.3375,0
1,1,0.9167,1,151.55,1
2,1,2,1,151.55,0
3,1,30,1,151.55,1
4,1,25,1,151.55,0
...,...,...,...,...,...
1304,3,14.5,1,14.4542,0
1305,3,0,1,14.4542,0
1306,3,26.5,0,7.225,1
1307,3,27,0,7.225,1


In [20]:
X_train, X_test, y_train, y_test = train_test_split(
    new_data, y, random_state=1)

In [22]:
y_train.count()

981

In [23]:
X_train.count()

pclass      981
age         981
sibsp       981
fare        981
sex_male    981
dtype: int64

In [24]:
X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# should be del
y_train_categorical = to_categorical(y_train)
y_train_categorical

# should be del
y_test_categorical = to_categorical(y_test)
y_test_categorical

In [25]:
from sklearn.linear_model import LogisticRegression
model_LogisticRegression = LogisticRegression()
model_LogisticRegression 

LogisticRegression()

In [26]:
## Question should y be encoded_y
## if 0 0r 1 then do not apply endoing
#model_LogisticRegression.fit(X_train_scaled , encoded_y)
model_LogisticRegression.fit(X_train_scaled , y_train)

LogisticRegression()

In [27]:
print(f"Training Data Score: {model_LogisticRegression.score(X_train_scaled, y_train)}")
print(f"Testing Data Score: {model_LogisticRegression.score(X_test_scaled, y_test)}")

Training Data Score: 0.7716615698267074
Testing Data Score: 0.8109756097560976


Make predictions

In [28]:
predictions = model_LogisticRegression.predict(X_test)
pd.DataFrame({"Prediction": predictions, "Actual": y_test})

,Prediction,Actual
201,0,0
115,1,0
255,1,1
1103,1,0
195,1,1
...,...,...
34,0,0
85,1,1
984,1,1
156,0,0


# Hyperparameter Tuning

Use `GridSearchCV` to tune the model's parameters

In [ ]:
# Create the GridSearchCV model

In [ ]:
# Train the model with GridSearch

In [ ]:
print(grid2.best_params_)
print(grid2.best_score_)

# Save the Model

In [ ]:
# save your model by updating "your_name" with your name
# and "your_model" with your model variable
# be sure to turn this in to BCS
# if joblib fails to import, try running the command to install in terminal/git-bash
import joblib
filename = 'your_name.sav'
joblib.dump(your_model, filename)